In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1591107246155_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
spark = SparkSession.builder.getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df_cow = spark.read.parquet('s3://data-eng-capstone-cf/production/dim_countries_of_the_world')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
df_cow.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

227

In [11]:
df_cow.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- country: string (nullable = true)
 |-- region: string (nullable = true)
 |-- population: integer (nullable = true)
 |-- gdp_dol_per_capita: integer (nullable = true)

In [10]:
df_cow.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+--------------------+----------+------------------+
|          country|              region|population|gdp_dol_per_capita|
+-----------------+--------------------+----------+------------------+
|      Afghanistan|ASIA (EX. NEAR EAST)|  31056997|               700|
|          Albania|      EASTERN EUROPE|   3581655|              4500|
|          Algeria|     NORTHERN AFRICA|  32930091|              6000|
|   American Samoa|             OCEANIA|     57794|              8000|
|          Andorra|      WESTERN EUROPE|     71201|             19000|
|           Angola|  SUB-SAHARAN AFRICA|  12127071|              1900|
|         Anguilla| LATIN AMER. & CARIB|     13477|              8600|
|Antigua & Barbuda| LATIN AMER. & CARIB|     69108|             11000|
|        Argentina| LATIN AMER. & CARIB|  39921833|             11200|
|          Armenia| C.W. OF IND. STATES|   2976372|              3500|
|            Aruba| LATIN AMER. & CARIB|     71891|             28000|
|     

In [6]:
df_fact = spark.read.parquet('s3://data-eng-capstone-cf/production/fact_i94')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
df_fact.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

740595

In [8]:
df_fact.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- i94bir: integer (nullable = true)
 |-- visatype: string (nullable = true)
 |-- visitor_count: long (nullable = true)
 |-- country: string (nullable = true)
 |-- country_fk: string (nullable = true)
 |-- i94mode: string (nullable = true)
 |-- i94visa: string (nullable = true)
 |-- i94mon: integer (nullable = true)
 |-- i94res: string (nullable = true)

In [9]:
df_fact.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+-------------+-----------+------------+------------+--------+------+-----------+
|i94bir|visatype|visitor_count|    country|  country_fk|     i94mode| i94visa|i94mon|     i94res|
+------+--------+-------------+-----------+------------+------------+--------+------+-----------+
|    56|      WT|         1091|SOUTH KOREA|Korea, South|         Air|Pleasure|     4|SOUTH KOREA|
|    69|      WB|           20|SOUTH KOREA|Korea, South|         Air|Business|     4|SOUTH KOREA|
|    56|      B1|           58|SOUTH KOREA|Korea, South|         Air|Business|     4|SOUTH KOREA|
|    78|     GMT|           20|SOUTH KOREA|Korea, South|         Air|Pleasure|     4|SOUTH KOREA|
|    39|     GMT|            3|SOUTH KOREA|Korea, South|Not reported|Pleasure|     4|SOUTH KOREA|
+------+--------+-------------+-----------+------------+------------+--------+------+-----------+
only showing top 5 rows

In [21]:
df_analysis = df_fact.filter(~df_fact.i94res.contains('MEXICO')) \
                     .groupBy('i94res') \
                     .sum('visitor_count') \
                     .orderBy(F.desc('sum(visitor_count)'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_analysis.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------+------------------+
|        i94res|sum(visitor_count)|
+--------------+------------------+
|UNITED KINGDOM|           4587092|
|         JAPAN|           3603786|
|    CHINA, PRC|           3049942|
|       GERMANY|           2046288|
|   SOUTH KOREA|           1982516|
|        BRAZIL|           1725479|
|        FRANCE|           1641152|
|     AUSTRALIA|           1350508|
|         INDIA|           1206771|
|         ITALY|            988860|
|     ARGENTINA|            926321|
|      COLOMBIA|            852618|
|         SPAIN|            807447|
|   NETHERLANDS|            674544|
|     VENEZUELA|            589848|
|        SWEDEN|            560581|
|   SWITZERLAND|            471070|
|        TAIWAN|            466267|
|       IRELAND|            443005|
|        ISRAEL|            425085|
+--------------+------------------+
only showing top 20 rows